In [133]:
from abc import ABC, abstractmethod
from collections.abc import MutableSet, Sequence

# Abstract Classes and Multiple Inheritance

In many of the examples we use to look at the concept of inheritance we have base classes like Animal or Vehicle - classes that capture all the common attributes and functionality of the objects we need, but classes that are not meant to be instantiated directly. There is no way to create a new Animal object, it doesn't make sense; we need to inherit from that Animal class into a Dog or Tiger before we can create an object that makes sense.

This is a very common pattern in object-oriented programming, and Python supports it by allowing us to define abstract classes. An abstract class is a class that cannot be instantiated, and is defined purely to be inherited into subclasses that can be instantiated.

## Abstract Methods

In an abstract class we can define abstract methods. An abstract method is a method that is declared in the abstract class, but contains no implementation. The implementation is provided by the subclass that inherits from the abstract class, and that implementation must be provided before the subclass can be instantiated. For example, an Animal class might have some method to reproduce(), but each subclass will have a totally different implementation - some will lay some eggs, others will birth a litter of kittens. Each type of animal needs to reproduce for it to exist, but they must each say how they will on their own. 

In this context, methods that are not abstract (i.e. they have an implementation) are called concrete methods. In an abstract class we can have both abstract and concrete methods, so we can save on duplication while still forcing subclasses to implement the methods that are necessary for them to exist. In the example below, we can create the calculate_gpa method that needs to be implemented in a child class, along with regular concrete methods that will not change depending on child class. The @abstractmethod decorator is used to mark a method as abstract, or to apply this rule that the method must be implemented in a child class.

In [134]:
class Personal_Transcript(ABC):

    def __init__(self, student_id):
        self._grades = {}
        self._student_id = student_id
    
    def get_ID(self):
        return self._student_id
    def add_grade(self, course, grade):
        self._grades[course] = grade
    
    @abstractmethod
    def list_passing(self):
        pass
    @abstractmethod
    def calculate_pass(self):
        pass

class Nine_Point_Transcript(Personal_Transcript):
    def __init__(self, student_id):
        super().__init__(student_id)

    
    def calculate_pass(self):
        vals = self._grades.values()
        gpa = sum(vals) / len(vals) 
        if gpa >= 5:
            return True
        else:
            return False
    
    def list_passing(self):
        '''Returns a list of courses with passing grades'''
        pass

class Four_Point_Transcript(Personal_Transcript):
    pass

In [135]:
ninePtTrans = Nine_Point_Transcript(123)
ninePtTrans.add_grade('Math', 8)
ninePtTrans.add_grade('English', 7)
ninePtTrans.add_grade('History', 9)
print(ninePtTrans.calculate_pass())

True


## Exercise

Implement list_passing above. 

Then add a class for Four_Point_Transcript that is equivalent. 

### ABC Class

In Python, abstract classes are defined using the ABC class from the abc module. ABC stands for Abstract Base Class, and it is a special class that allows us to define abstract methods. We can inherit from ABC to create an abstract class, and we can use the @abstractmethod decorator to mark methods as abstract. We can also use the @abstractclassmethod and @abstractstaticmethod decorators to mark class methods and static methods as abstract. In practice, this is a simple thing to implement - if we want to make an abstract class, we inherit from ABC and mark the methods we want to be abstract with the @abstractmethod decorator.

## Abstract Collections 

One of the most common uses of abstract classes is to use them to create new data structures. For example, we might want to create a new type of list or dictionary that is tailored to what we need. To do this, we can inherit from one of the abstract base classes that is the closest base to what we need. In Python, our data structures tend to be interchangeable in many scenarios, and this inheritance hierarchy helps to facilitate that. At its core any data structure is simple - a bucket of data and some methods to manipulate that data. Each variation of a data structure is just a different arrangement of these two basic ideas. In line with the idea of duck typing, or something being "good enough" if it behaves like what we need, we can use the abstract base classes to create new data structures that are inherently usable wherever we may use the built-in data structures, combined with our own custom functionality.

Each of the base classes is best suited for certain types of data storage, so in most cases we can pick something that is close to what we need, then just add a few modifications to tailor it to our needs:

![ABC Collections](../../images/abc_collections.png "ABC Collections")
![ABC Collections](../images/abc_collections.png "ABC Collections")

The base classes provide some functionality that is common for those types, the most common ones being the sequence ones for list-ish data structures and hashable ones for dictionary-ish data structures. There is a full list of the abstract base classes in the [Python documentation](https://docs.python.org/3/library/collections.abc.html). In that list, the abstract method column lists the ones that you must override in your subclass for things to work. 

<b>Note:</b> it is normally a better choice to inherit from the abstract base classes for data structures rather than to try to inherit from the built-in Python data structures directly. The abstract base classes are designed to be inherited from, while the built-in data structures are optimized for efficiency. You can inherit from the built-in types, but there may be scenarios where you get weird behavior or where you have to override a lot of functionality to get the behavior you want.

### Example - List Based Set

For an example, we can make a new type of set that is based on a list. To do this, we need to define all of the abstract methods that are required for a mutable set:
<ul>
<li> <b>__contains__</b> - to check if an item is in the set</li>
<li> <b>__iter__</b> - to iterate over the items in the set</li>
<li> <b>__len__</b> - to get the number of items in the set</li>
<li> <b>add</b> - to add an item to the set</li>
<li> <b>discard</b> - to remove an item from the set</li>
</ul>

All the other functionality is inherited from the mutable set base class and will work as expected. We can also override any of the other methods if we want to change the behavior. For example, we could override the __str__ method to print the set in a different format. In particular, you can see in our example <b><i>we didn't provide the __and__ implementation, we get that as a bonus from the base class</i></b>. 

In [136]:
class ListBasedSet(MutableSet):
    ''' Alternate set implementation favoring space over speed
        and not requiring the set elements to be hashable. '''
    def __init__(self, iterable):
        self.elements = []
        for value in iterable:
            if value not in self.elements:
                self.elements.append(value)

    def __iter__(self):
        return iter(self.elements)

    def __contains__(self, value):
        return value in self.elements

    def __len__(self):
        return len(self.elements)
    def __str__(self) -> str:
        return self.elements.__str__()
    
    def add(self, value):
        if value not in self:
            self.elements.append(value)
    def discard(self, value) -> None:
        self.elements.remove(value)   

In [137]:
s1 = ListBasedSet('abcdef')
s2 = ListBasedSet('defghi')
overlap = s1 & s2
print(overlap)

['d', 'e', 'f']


## Minimal Multiple Inheritance

In Python we are able to do multiple inheritance, where a class inherits from more than one base class. This is a very powerful feature, but it can also be a source of problems. In particular, if we have two base classes that have the same method name, which one will be used? The idea motivating multiple inheritance fits well if we really try to model scenarios in terms of objects and their relationships. 

For example, we might have a class that represents a student, and another class that represents a teacher. Both of these classes might have a method called teach(), but they will do very different things. If we have a class that represents a teaching assistant, it might make sense for it to inherit from both the student and teacher classes. In this case, the teaching assistant will have both a teach() method that comes from the student class and a teach() method that comes from the teacher class.

Multiple inheritance can allow us to again minimize duplication and implement functionality only once, where it "lives". Other objects can be "one of" lots of different parent classes, inheriting functionality from all of them.

## Method Resolution Order

When we have multiple inheritance, we need to know which method will be used when we call a method on an object. This is called the method resolution order, and it is determined when the class is created. The method resolution order is the order in which the base classes are searched for a method implementation. The method resolution order is determined by the order in which the base classes are listed in the class definition. For example, if we have a class that inherits from two base classes, the first base class listed will be searched first, then the second base class, then the class itself.

### Problems with Multiple Inheritance

Multiple inheritance can be a very powerful tool, but it can also be a source of problems, mainly in making code confusing and hard to understand. In particular, if we have a class that inherits from two base classes, and those base classes have a method with the same name, which one will be used? This can be confusing, and it can be hard to debug.

For us, multiple inheritance is something to be aware of, but not something we'll invest much time or effort on. For data science applications it isn't something that comes up all that frequently, and cases where it may be useful tend to be scenarios that are more complex than what we are doing, such as when we are integrating data pipelines into a larger software system.

![Multiple Inheritance](../../images/multiple_inh.png "Multiple Inheritance")
![Multiple Inheritance](../images/multiple_inh.png "Multiple Inheritance")

#### Example - Multiple Inheritance

The example below shows a class that inherits from two base classes, the MutableSet that provides the set functionality, and the Sequence class that provides the list-like functionality. I'll also define my own weird __str__ method to show that we can override any of the methods we want to. This particular class should work fine, as the general setup of the two base classes is compatible. I don't think we'll run into any issues with conflicting method names. If we do run into one, the MutableSet class will take precedence over the Sequence class, as it is listed first in the class definition.

In [138]:
class ListBasedSet2(MutableSet, Sequence):

    def __init__(self, iterable):
        self.elements = []
        for value in iterable:
            if value not in self.elements:
                self.elements.append(value)

    def __iter__(self):
        return iter(self.elements)

    def __contains__(self, value):
        return value in self.elements

    def __len__(self):
        return len(self.elements)
    def __str__(self) -> str:
        return self.elements.__str__()
    
    def __getitem__(self, index):
        return self.elements[index]
    
    def __str__(self) -> str:
        tmp_str = "<"
        for item in self.elements:
            tmp_str += str(item) + " * "
        tmp_str += ">"
        return tmp_str
    
    def add(self, value):
        if value not in self:
            self.elements.append(value)
    def discard(self, value) -> None:
        self.elements.remove(value) 

In [139]:
lbs2 = ListBasedSet2('abcdef')
print(lbs2)

<a * b * c * d * e * f * >


## Exercise

In [140]:
# Create a silly class that inherits from one of the abstract base classes in the collections module.
# Also inherit from this function. 

In [141]:
class Summer(ABC):
    
    @abstractmethod
    def getSum(self, cutoff=0):
        '''Returns the sum of the elements in the collection, only if the value of each item is over the cutoff'''
        pass


In [142]:
class SetSummer(Summer, Sequence):
    pass

In [143]:
sum1 = SetSummer([1,2,3,4,5,6,7,8,9,10], cutoff=5)
print(sum1.getSum())

40
